<a href="https://colab.research.google.com/github/JJ-HMFIC/ML_Basic/blob/main/ML_Basic_Ch11_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

11-3-mnist_cnn_class

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical

# hyper parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

class Model(tf.keras.Model):

    def __init__(self):
        super(Model, self).__init__()
        self.keep_prob = 0.7

        # L1 # build_net 부분은 생성자에 포함
        self.conv1 = layers.Conv2D(32, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu', input_shape=(28, 28, 1))
        self.pool1 = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')
        self.dropout1 = layers.Dropout(self.keep_prob)

        # L2
        self.conv2 = layers.Conv2D(64, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')
        self.pool2 = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')
        self.dropout2 = layers.Dropout(self.keep_prob)

        # L3
        self.conv3 = layers.Conv2D(128, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')
        self.pool3 = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')
        self.dropout3 = layers.Dropout(self.keep_prob)

        # L4
        self.flatten = layers.Flatten()
        self.dense4 = layers.Dense(625, activation='relu')
        self.dropout4 = layers.Dropout(self.keep_prob)

        # L5
        self.dense5 = layers.Dense(10, activation='softmax')

    def call(self, inputs, training=False):
        x = self.conv1(inputs)
        x = self.pool1(x)
        x = self.dropout1(x, training=training)

        x = self.conv2(x)
        x = self.pool2(x)
        x = self.dropout2(x, training=training)

        x = self.conv3(x)
        x = self.pool3(x)
        x = self.dropout3(x, training=training)

        x = self.flatten(x)
        x = self.dense4(x)
        x = self.dropout4(x, training=training)

        return self.dense5(x)

# create model
model = Model()

# compile the model
model.compile(optimizer=tf.optimizers.Adam(learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

# preprocess the data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float32')
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1).astype('float32')

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# train the model
model.fit(x_train, y_train, epochs=training_epochs, batch_size=batch_size)

# evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test Accuracy:', test_acc)


Epoch 1/15
600/600 [==============================] - 5s 6ms/step - loss: 0.8857 - accuracy: 0.6974
Epoch 2/15
600/600 [==============================] - 4s 7ms/step - loss: 0.3301 - accuracy: 0.8957
Epoch 3/15
600/600 [==============================] - 3s 6ms/step - loss: 0.2607 - accuracy: 0.9182
Epoch 4/15
600/600 [==============================] - 4s 7ms/step - loss: 0.2303 - accuracy: 0.9297
Epoch 5/15
600/600 [==============================] - 7s 11ms/step - loss: 0.2118 - accuracy: 0.9342
Epoch 6/15
600/600 [==============================] - 6s 10ms/step - loss: 0.1995 - accuracy: 0.9396
Epoch 7/15
600/600 [==============================] - 5s 8ms/step - loss: 0.1887 - accuracy: 0.9420
Epoch 8/15
600/600 [==============================] - 7s 11ms/step - loss: 0.1838 - accuracy: 0.9444
Epoch 9/15
600/600 [==============================] - 4s 7ms/step - loss: 0.1750 - accuracy: 0.9470
Epoch 10/15
600/600 [==============================] - 3s 6ms/step - loss: 0.1683 - accuracy: 0.9

앙상블(Ensemble)"은 여러 개의 모델을 결합하여 하나의 모델로 사용하는 기법을 의미합니다.

이는 여러 모델의 예측을 조합하여 더 강력하고 안정적인 예측을 만들어내는 방법입니다

In [ ]:
test_size = len(mnist.test_labels)
predictions = np.zeros([test_size, 10])
for m_idx, m in enumerate(models):
    print(m_idx, 'Accuracy:', np.mean(np.argmax(m.predict(mnist.test_images, training=False), axis=1) == np.argmax(mnist.test_labels, axis=1)))
    p = m.predict(mnist.test_images, training=False)
    predictions += p.numpy()

ensemble_correct_prediction = np.equal(np.argmax(predictions, 1), np.argmax(mnist.test_labels, 1))
ensemble_accuracy = np.mean(ensemble_correct_prediction)
print('Ensemble accuracy:', ensemble_accuracy)